In [1]:
require 'nn'
require 'rnn'

In [2]:
-- Some useful functions
function genNbyK(n, k, a, b)
    out = torch.LongTensor(n, k)
    for i=1, n do
        for j = 1, k do
            out[i][j] = torch.random(a, b)
        end
    end
    return out
end

function buildModel(model, vocabSize, embeddingSize, metric, adapt, use_cuda)
    -- Small experiments seem to show that the Tanh activations performed better\
    --      than the ReLU for the bow model
    if model == 'bow' then
        print(string.format("Running bag-of-words model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.Sum(2, 3, true)) -- Not averaging blows up model so keep this true
                    :add(nn.Tanh())
    else
        print(string.format("Running LSTM model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.SplitTable(2))
                    :add(nn.Sequencer(nn.LSTM(embeddingSize, embeddingSize)))
                    :add(nn.SelectTable(-1))            -- selects last state of the LSTM
                    :add(nn.Linear(embeddingSize, embeddingSize))
                    :add(nn.ReLU())
    end
    local queryLookup = sentenceLookup:clone("weight", "gradWeight") 
    local summaryLookup = sentenceLookup:clone("weight", "gradWeight")
    local pmodule = nn.ParallelTable()
                :add(sentenceLookup)
                :add(queryLookup)
                :add(summaryLookup)

    if model == 'bow' then
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.Tanh())
            :add(nn.Linear(embeddingSize * 3, 2))
    else
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.ReLU())
            :add(nn.Linear(embeddingSize * 3, 2))
    end

    if adapt then 
        print("Adaptive regularization")
        local logmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 1))
            :add(nn.LogSigmoid())
            :add(nn.SoftMax())

        local regmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 2))

        local fullmod = nn.ConcatTable()
            :add(regmod)
            :add(logmod)

        local final = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(fullmod)

        nnmodel = final
    end

    if use_cuda then
        return nnmodel:cuda()
    end
    return nnmodel
end

In [3]:
-- Setting parameters
n = 10
n_s = 5
k = 15
q = 5
a = 1
b = 100
embDim = 50
SELECT = 2
SKIP = 1

In [4]:
-- Simulating streams and queries
queries = genNbyK(n, q, a, b)

-- Note that the sentences are batched by sentence index so sentences[1] is the first sentence of each article
sentences = {}
for i=1, n_s do
    sentences[i] = genNbyK(n, k, a, b)
end

-- Using this to generate the optimal actions
true_actions = {}
for i=1, n_s do 
    ---- Simulating the data
    trueqValues = torch.rand(n, 2)                   
     ---- Generating the max values and getting the indices
    qMaxtrue, qindxtrue = torch.max(trueqValues, 2) 
    --- I want to select the qindx elements for each row
    true_actions[i] = torch.zeros(n, 2):scatter(2, qindxtrue, torch.ones(trueqValues:size()))
end

In [5]:
model = buildModel('bow', b, embDim, 'f1', false, false)

Running bag-of-words model to learn f1	


In [20]:
preds = model:forward({sentences, queries, torch.zeros(n, q)})
print(preds)

 0.0518  0.0576
 0.0192  0.2835
 0.1888  0.0355
 0.0983  0.0208
 0.0706  0.1803
-0.0029  0.2815
-0.0268 -0.1170
 0.0811 -0.0627
-0.1178  0.0416
-0.0486  0.0576
[torch.DoubleTensor of size 10x2]



In [33]:
qMax, qindx = torch.max(preds, 2)
actions = torch.zeros(n, 2):scatter(2, qindx, torch.ones(preds:size()))

In [6]:
predsummary = torch.zeros(n, 150)

In [8]:
sentences[1]

 12  29  47  48  13  67   5  19  79  47  54  61  74   3  72
 35  24  15  99  46  33  27  98  30  33  24  97  14  63  18
 65  27  22  52  84  60  76  61  68  76  28  20  78   4  70
 28  98  56  23  53  49  56  77  96  60  16  73  42  27   9
 66  31  28  40  82  83  45  37  33  35  63  64  51  80  92
 54  46  37   7  96   5  17   1  21  55   2  46  31  38  95
 55  91  71  29  33  94  81  33  72  64  90  25  32  54  38
 28  79  94  73  73  26  54  73  12  35  83  16  69  10  30
 44  81  64  39  97  48  48  95  34  46  23  15   4  59  64
 17  64  47  34   5  58  72  51  50  28  90  25  94  74  22
[torch.LongTensor of size 10x15]



In [104]:
indices = torch.linspace(1, x:size(1), x:size(1) ):long()
selected = indices[x:eq(1)]

In [105]:
selected

  1
  2
  5
  6
  9
 10
[torch.LongTensor of size 6]



In [91]:
x = actions:select(2, SELECT)
sb = sentences[1]

In [ ]:
x, sb

In [98]:
sentences

{
  1 : LongTensor - size: 10x15
  2 : LongTensor - size: 10x15
  3 : LongTensor - size: 10x15
  4 : LongTensor - size: 10x15
  5 : LongTensor - size: 10x15
}


In [39]:
indx = torch.range(1, 10)
-- Summary index for selecting the words
sumindx = nn.MaskedSelect():forward({indx, actions:select(2,1):resize(10, 1):byte()})  --- sentences
-- need a LongTensor to do the selection
-- sumindx = torch.LongTensor(sumindx:size(1)):copy(sumindx)

In [ ]:
indx = torch.range(1, 10)
-- Summary index for selecting the words
sumindx = nn.MaskedSelect():forward({indx, actions:select(2,1):resize(10, 1):byte()})  --- sentences
-- need a LongTensor to do the selection
sumindx = torch.LongTensor(sumindx:size(1)):copy(sumindx)
--- This gives the wrong selection we want
print(sentences:index(1, sumindx))